In [ ]:
import time
import collections
import Leap
from Leap import Bone

import csv

'''
Gets the current frame from controller
for each finger, stores the topmost end of each bone (4 points)
adjusts bone location relativity by subtracting the center of the palm
returns the adjusted bone locations in the form:
{feat0=some_float, feat1=some_float, ... feat59=some_float}
'''

def get_hand_position(controller, blocking=False):
    cols_to_norm = ['feat1', 'feat2', 'feat3', 'feat4', 'feat5', 'feat6', 'feat7', 'feat8',
       'feat9', 'feat10', 'feat11', 'feat12', 'feat13', 'feat14', 'feat15',
       'feat16', 'feat17', 'feat18', 'feat19', 'feat20', 'feat21', 'feat22',
       'feat23', 'feat24', 'feat25', 'feat26', 'feat27', 'feat28', 'feat29',
       'feat30', 'feat31', 'feat32', 'feat33', 'feat34', 'feat35', 'feat36',
       'feat37', 'feat38', 'feat39', 'feat40', 'feat41', 'feat42', 'feat43',
       'feat44', 'feat45', 'feat46', 'feat47', 'feat48', 'feat49', 'feat50',
       'feat51', 'feat52', 'feat53', 'feat54', 'feat55', 'feat56', 'feat57',
       'feat58', 'feat59', 'feat60']
    
    # Get the current frame
    frame = controller.frame()
    
    if not blocking and len(frame.fingers) == 0:
        return None
    while len(frame.fingers) == 0:
        frame = controller.frame()

    fingers = controller.frame().fingers
    finger_bones = []
    for finger in fingers:
        finger_bones.append(finger.bone(Bone.TYPE_METACARPAL).next_joint)
        finger_bones.append(finger.bone(Bone.TYPE_PROXIMAL).next_joint)
        finger_bones.append(finger.bone(Bone.TYPE_INTERMEDIATE).next_joint)
        finger_bones.append(finger.bone(Bone.TYPE_DISTAL).next_joint)
    
    # Possible issue when more than one hand
    hands = controller.frame().hands
    hand_center = 0
    for hand in hands:
        hand_center = hand.palm_position

    # Calibrated_finger_bones = collections.OrderedDict()
    calibrated_finger_bones = []
    for i in range(len(finger_bones)):
        normalized_joint = (finger_bones[i] - hand_center).to_tuple()
        for j in range(3):
            calibrated_finger_bones.append(normalized_joint[j])
    
    # Open a CSV file and write the data
    csv_header = list(range(1, 61))
    file_handle = open("data_1.csv", "w")
    csv_writer = csv.DictWriter(file_handle, csv_header)
    data_dict = {}
    for index, value in enumerate(cols_to_norm, 1):
        data_dict[index] = value
    csv_writer.writerow(data_dict)
    file_handle.close()

    # Open a CSV file and write the header
    csv_header = list(range(1, 61))
    file_handle = open("data_1.csv", "a+")
    csv_writer = csv.DictWriter(file_handle, csv_header)
    data_dict = {}
    for index, value in enumerate(calibrated_finger_bones, 1):
        data_dict[index] = value
    csv_writer.writerow(data_dict)
    
    file_handle.close()


if __name__ == "__main__":

    controller = Leap.Controller()

    while True:
        get_hand_position(controller)
        time.sleep(2)